In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers, losses
from google.colab import drive
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import numpy as np
import random
import math
drive.mount('/content/drive')
os.chdir('/content/drive/Shareddrives/ML_final_team01')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
directory = 'female-crop-train-png'
file_dir = os.listdir(directory)
random.shuffle(file_dir)
file_dir = file_dir[:2000]
print(len(file_dir))
X_train = np.array([])
X_train = np.zeros((len(file_dir), 216, 334, 3))
for p in range(len(file_dir)):
    img = cv2.imread(directory+"/"+file_dir[p])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255
    X_train[p] = np.reshape(img, (216, 334, 3))

2000


In [ ]:
female_train_csv = pd.read_csv("female_train.csv")
# convert label to one-hot encoded
y_train = np.zeros((len(file_dir), 4))
for i in range(len(file_dir)):
    #print(file_dir[i])
    row = female_train_csv.loc[female_train_csv['filename'] == 'cv-other-train/'+file_dir[i][:-3]+'mp3']
    # print(row['age'].values[0])
    if row['age'].values[0] == 'teens' or row['age'].values[0] == 'twenties':
        y_train[i][0] = 1
    elif row['age'].values[0] == 'thirties' or row['age'].values[0] == 'fourties':
        y_train[i][1] = 1
    elif row['age'].values[0] == 'fifties' or row['age'].values[0] == 'sixties':
        y_train[i][2] = 1
    elif row['age'].values[0] == 'seventies' or row['age'].values[0] == 'eighties' or row['age'].values[0] == 'nineties':
        y_train[i][3] = 1
print(y_train)

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]


In [ ]:

# get validation data
directory = 'female-crop-test-png'
file_dir = os.listdir(directory)
random.shuffle(file_dir)
file_dir = file_dir[:500]
#print(file_dir)
X_test = np.array([])
X_test = np.zeros((len(file_dir), 216, 334, 3))
for p in range(len(file_dir)):
    img = cv2.imread(directory+"/"+file_dir[p])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255
    X_test[p] = np.reshape(img, (216, 334, 3))


In [ ]:
female_test_csv = pd.read_csv("female_test.csv")
# convert label to one-hot encoded
y_test = np.zeros((len(file_dir), 4))
for i in range(len(file_dir)):
    #print(file_dir[i])
    row = female_test_csv.loc[female_test_csv['filename'] == 'cv-other-test/'+file_dir[i][:-3]+'mp3']
    # print(row['age'].values[0])
    if row['age'].values[0] == 'teens' or row['age'].values[0] == 'twenties':
        y_test[i][0] = 1
    elif row['age'].values[0] == 'thirties' or row['age'].values[0] == 'fourties':
        y_test[i][1] = 1
    elif row['age'].values[0] == 'fifties' or row['age'].values[0] == 'sixties':
        y_test[i][2] = 1
    elif row['age'].values[0] == 'seventies' or row['age'].values[0] == 'eighties' or row['age'].values[0] == 'nineties':
        y_test[i][3] = 1
print(y_test)

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (4, 4), strides = 1, activation = 'relu', input_shape = (216, 334, 3)))
model.add(layers.MaxPooling2D((4, 4), strides = 2))
model.add(layers.Conv2D(32, (4, 4), activation='relu', strides = 2))
model.add(layers.MaxPooling2D((4, 4), strides = 2))
model.add(layers.Conv2D(64, (4, 4), activation='relu', strides = 2))
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))
#model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
History = model.fit(X_train, y_train, batch_size = 100, epochs = 10, validation_data = (X_test, y_test))

Epoch 1/10


AttributeError: ignored

In [ ]:
# model.save('female-age-pred')

In [ ]:
X_predict = model.predict(X_test[2:3])
age = ['teens', 'twenties', 'thirties', 'fourties', 'fifties', 'sixties', 'older than seventies']
outcome = (-X_predict[0]).argsort()[:2]
print(round(X_predict[0][outcome[0]] * 100, 1), "% from", age[outcome[0]])
print(round(X_predict[0][outcome[1]] * 100, 1), "% from", age[outcome[1]])


# print((X_predict))

In [ ]:
# model.save('female-age-pred')

In [ ]:
f1 = custom_F1score_loss.call(None, y_test, X_predict)
print(f'The f1 score = {f1}')